In [2]:
import pandas as pd
import numpy as np
import re
import unicodedata
import pickle

In [92]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 2000)

In [93]:
data = pd.read_table('text_files/football-city-html.txt') 

In [94]:
df = data.rename(columns={"</div>": "data"})

In [95]:
terms = ['col-sm-6 col-xs-6', 'col-sm-3 hidden-xs', '/modules/images/flags/']

In [96]:
df = df[df.data.str.contains('|'.join(terms))] 

In [97]:
df.drop(index=5, inplace=True)

In [98]:
df.shape

(5782, 1)

In [99]:
pattern = '|'.join(['<div class=\"col-sm-6 col-xs-6\"><b>', '<img src=\"/modules/images/flags/_', '<div class=\"col-sm-3 hidden-xs\">', '</b>', '</div>'])

In [100]:
df.data = df.data.str.replace(pattern, '')

<ipython-input-100-c1bbdffa1003>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.data = df.data.str.replace(pattern, '')


In [101]:
def removing_patterns(df: pd.DataFrame, col_names: list, patern: str):
    df[col_names].apply(lambda x: str(x) if re.search(pattern, str(x)) is None else re.findall(pattern, str(x)))
    return df

In [102]:
pattern = r'\"[\s|\w]+\">$'
df = removing_patterns(df, list(df), pattern)

In [103]:
def find_no_utf_symbols(sr: pd.Series):
    find_symols = r'[^.a-zA-Z0-9)(&"\s>;<=,-/}{\']'
    rare_symbols = sr.map(lambda x: ''.join(re.findall(find_symols, str(x))))
    return rare_symbols

In [104]:
rare_symbols = find_no_utf_symbols(df.data)

In [105]:
str(set(rare_symbols))

"{'', 'ăăş', 'â', 'čŠ', 'å', 'əəə', 'žíš', 'ôô', 'ľ', 'óá', 'ı', 'İ', 'ığ', 'ýŽ', 'šć', 'áž', 'ľý', 'àò', 'çöüü', 'ź', 'Şüə', 'ă', 'ññ', 'ūẓī', 'Š', 'ýáš', 'âş', 'ń', 'È', 'ř', 'ș', 'ô', 'ış', 'óŁę', 'ā’', 'ç', 'ø', 'àộ', 'ëçë', 'š', 'î', 'éšě', 'ľč', 'ěříž', 'šňé', 'ãã', 'éë', 'çğ', 'Ñú', 'éé', 'ıçğ', 'áš', 'éḥ', 'ééá', 'ří', 'ööö', 'ö', 'áé', 'íð', 'ýá', 'Šľ', 'ţ', 'íá', '’éé', 'Şı', 'ë', 'łł', 'ė', 'ʿā', 'ïé', 'ðæ', 'áŠ', 'áá', 'áí', 'šá', 'Žž', 'çã', 'ď', 'ýí', 'çş', 'āḍ', 'ş', 'ġā', 'ū', 'žá', 'čí', 'İÖİ', 'è', 'äé', 'Čč', 'đ', 'É', 'öä', 'íí', 'ää', 'Ħ', 'ê', 'ėž', 'óę', 'ăţ', 'óú', 'ġāī', 'øø', 'é', 'ž', 'ğ', 'öð', 'šýš', 'Łę', 'Ō', 'łó', 'õ', 'ą', 'ãí', 'íš', 'áň', 'ží', 'ć', 'Í', 'âá', 'šéá', 'ăăăş', 'ëë', 'óó', 'ủầộ', 'ðø', 'ýŽáš', 'žň', 'í', 'ā', 'áó', 'əə', 'čš', 'éó', 'üü', 'çë', 'ľé', 'Łóź', 'æ', 'Ž', 'úí', 'ę', '’', 'ä', 'á', 'ăă', 'ã', 'ãç', 'şş', 'éá', 'Śś', 'º', 'ï', 'à', 'íáé', 'ú', 'ñ', 'éï', 'ó', 'óñ', 'ī', 'óã', 'ň', 'Å', 'šáá', 'č', 'ęó', 'ü', 'öö', 'îş', 'ō', 'á

In [106]:
all_symbols = []
for symbol in rare_symbols:
    symbols = ''.join(symbol)
    all_symbols.append(symbols)
all_symbols = str(all_symbols)
all_symbols = ''.join(set(str(all_symbols)))
all_symbols = all_symbols.replace(' ', '')
all_symbols = all_symbols.replace(']', '')
all_symbols = all_symbols.replace('[', '')
all_symbols = all_symbols.replace(',', '')
all_symbols = all_symbols.replace('\'', '')

In [107]:
all_symbols

'âåľÖòıİḍźăÈŠńřșôçýøšẓîðủöţəëėŚďşūèđÉĦêéžğŁŌõąćÍíāśæŽę’ḥäáầČãÑºġïàúñóīŞěňÅčüʿōộł'

In [108]:
pattern_2 = r'^[0-9]+.png'
pattern_3 = r'title=\"(.+?)\"'

In [109]:
df = df.data.apply(lambda x: ''.join(re.findall(pattern_3, str(x)) if re.search(pattern_2, str(x)) is not None else x))

In [110]:
df = pd.DataFrame(df)

In [111]:
df.shape

(5782, 1)

In [112]:
df.iloc[648:649,:]

,data
3067,


In [113]:
df.data.replace('', np.nan, inplace=True)

In [114]:
df.dropna(subset=['data'], inplace=True)

In [115]:
df.shape

(5757, 1)

In [116]:
def str_decode(str_obj: str):
    normalized = unicodedata.normalize('NFD', str_obj)
    decode_str = u"".join([c for c in normalized if not unicodedata.combining(c)])
    return decode_str

In [117]:
df.data = df.data.apply(lambda x: str_decode(x))

In [118]:
rare_symbols_keys = {'ł':'l','Ł':'L','º':'','ʿ':'\'','ə':'e','Ħ':'H','đ':'d','’':'\'','ı':'i','æ':'a','ð':'d','ø':'o'}

def replace_rare_symbols(df: pd.DataFrame, col_name: str, dict_obj: dict):
    for k, v in rare_symbols_keys.items():
        df[col_name] = df[col_name].apply(lambda x: x.replace(k, v) if re.search(k, str(x)) else x)
    return df[col_name]       

In [119]:
df.data = replace_rare_symbols(df, 'data', rare_symbols_keys)

In [121]:
df.head(3)

,data
13,AaB
18,Denmark
20,Aalborg


In [122]:
df.reset_index(inplace=True)

In [123]:
df.drop('index', axis=1, inplace=True)

In [124]:
club, country, city = df[0::3], df[1::3], df[2::3]

In [125]:
table_df = pd.concat([club.reset_index(drop=True), country.reset_index(drop=True), city.reset_index(drop=True)], axis=1)

In [126]:
table_df.columns.values[0] = "club"
table_df.columns.values[1] = "country"
table_df.columns.values[2] = "city"

In [127]:
table_df.shape

(1919, 3)

In [128]:
df = table_df.copy()

In [129]:
# take city.values from hooks for transcription
df.city = df.city.map(lambda x: ''.join(re.findall(r'\((.*?)\)', x)) if re.search(r'\(', x) else x)

In [130]:
# remove Buenos Aires districts
buenos_pattern = 'Buenos'
df.city = df.city.apply(lambda x: x if re.search(buenos_pattern, str(x)) is None else 'Buenos Aires')

In [131]:
# take city.values before '/'
df.city = df.city.map(lambda x: ''.join(re.findall(r'^(.*?)/', x)) if re.search(r'\/', x) else x)

In [132]:
# take city.values before ',' without districts
df.city = df.city.map(lambda x: ''.join(re.findall(r'^(.*?),', x)) if re.search(r',', x) else x)

In [133]:
# rename 'D.F.' at 'Mexico'
mexico_pattern = 'D.F.'
df.city = df.city.apply(lambda x: x if re.search(mexico_pattern, str(x)) is None else 'Mexico')

In [134]:
df.head()

,club,country,city
0,AaB,Denmark,Aalborg
1,Aalesund,Norway,Alesund
2,Aarau,Switzerland,Aarau
3,ABC,Brazil,Natal
4,Aberdeen,Scotland,Aberdeen


In [135]:
df.tail()

,club,country,city
1914,Yokohama,Japan,Yokohama
1915,Yokohama F. Marinos,Japan,Yokohama
1916,Young Africans,Tanzania,Dar-es-Salaam
1917,Young Boys,Switzerland,Bern
1918,Young Buffaloes,Swaziland,Manzini


### Add teams

In [64]:
with open('pickle_files/df_teams_data', 'rb') as f:
    df_for_add = pickle.load(f)

In [65]:
df_for_add.shape

(1934, 3)

In [60]:
club = 'Vila Nova'
country = 'Brazil'
city = 'Goiania'

new_club = {'club':club, 'club_country':country, 'club_city':city}
new_club = pd.DataFrame([new_club])
print(new_club)
df_for_add = pd.concat([df_for_add, new_club], ignore_index=True)

        club club_country club_city
0  Vila Nova       Brazil   Goiania


In [61]:
df_for_add.sort_values(by='club', ascending=True, inplace=True)
df_for_add.reset_index(drop=True, inplace=True)
df_for_add.shape

(1934, 3)

In [62]:
df_for_add[df_for_add.club == club]

,club,club_country,club_city
1842,Vila Nova,Brazil,Goiania


In [63]:
# with open('pickle_files/df_teams_data', 'wb') as f:
#     pickle.dump(df_for_add, f)